In [ ]:
import os
import urllib
import csv
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import concat
from pandas import DataFrame
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, LightSource
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn import datasets, linear_model
import scipy.io as sio
import datetime
from dateutil.relativedelta import relativedelta
import requests
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, timezone
from mintpy.utils import readfile
from mintpy.defaults.plot import *
from mintpy.objects.gps import search_gps, GPS
from mintpy.objects import sensor
from mintpy.utils import utils as ut
from mintpy.utils import readfile
from mintpy.view import prep_slice, plot_slice


%load_ext jupyter_ai

In [ ]:
def get_earthquakes(start_time, end_time, plot_box, depth_range="0 10", mag_range="0 10"):
    # Define the API endpoint and parameters
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    
    depth_max, depth_min = map(lambda x: -float(x), depth_range.split())
    
    params = {
       "format": "geojson",
       "starttime": start_time,
       "endtime": end_time,
       "minlatitude": plot_box[0],
       "maxlatitude": plot_box[1],
       "minlongitude": plot_box[2],
       "maxlongitude": plot_box[3],
       "mindepth": depth_min,   # Minimum depth (in kilometers)
       "maxdepth": depth_max,   # Maximum depth (in kilometers)
    }
    
    # Make a request to the USGS API
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extract earthquake information from the API response
    earthquakes = data["features"]
    earthquake_data = []

    # Calculate the Unix timestamp in milliseconds of start and end time
    min_time = int(datetime.strptime(start_time, "%Y-%m-%d").timestamp() * 1000)
    max_time = int(datetime.strptime(end_time, "%Y-%m-%d").timestamp() * 1000)

    for quake in earthquakes:
        magnitude = quake["properties"]["mag"]
        latitude = quake["geometry"]["coordinates"][1]
        longitude = quake["geometry"]["coordinates"][0]
        depth = quake["geometry"]["coordinates"][2]
        time = quake["properties"]["time"]
        
        earthquake_data.append([time,latitude, longitude, depth, magnitude])
    
    # Create a DataFrame from the earthquake data
    columns = ["Time", "Latitude", "Longitude", "Depth", "Magnitude"]
    events_df = pd.DataFrame(earthquake_data, columns=columns)
    return events_df

def normalize_earthquake_times(events_df, start_time, end_time):
# Normalize times for colormap (use the Unix timestamp in milliseconds)
    min_time = int(datetime.strptime(start_time, "%Y-%m-%d").replace(tzinfo=timezone.utc).timestamp() * 1000)
    max_time = int(datetime.strptime(end_time, "%Y-%m-%d").replace(tzinfo=timezone.utc).timestamp() * 1000)
    norm_times = [(time - min_time) / (max_time - min_time) for time in events_df["Time"]]
    return norm_times

def modify_colormap(cmap_name = "plasma_r", exclude_beginning = 0.15, exclude_end = 0.25, show = False):
    """ modify a colormap by excluding percentages at the beginning and end """

    cmap = plt.cm.get_cmap(cmap_name)
    cmap

    num_colors_to_exclude_beginning = int(len(cmap.colors) * exclude_beginning)
    num_colors_to_exclude_end = int(len(cmap.colors) * exclude_end)
    
    # Create a custom colormap by excluding the specified percentages of colors
    colors = cmap.colors[num_colors_to_exclude_beginning:-num_colors_to_exclude_end]
    cmap_custom = LinearSegmentedColormap.from_list('cmap_custom', colors, N=256)
    
    if show is True: 
        # Create a plot with the custom colormap
        data = [[0, 1, 2, 4], [1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 5, 6]]
        plt.imshow(data, cmap=cmap_custom)
        plt.colorbar()
        plt.show()

    return cmap_custom

def add_colorbar(cmap, start_time="", end_time=""):
    # Convert date strings to datetime objects
    start_time_date = datetime.strptime(start_time, "%Y-%m-%d")
    end_time_date = datetime.strptime(end_time, "%Y-%m-%d")
    
    # Create a separate colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=1))
    sm._A = []  # Hack to avoid normalization
    cbar = plt.colorbar(sm, ax=ax, shrink=0.5)  # Adjust the shrink value as needed
    
    # Set custom tick locations and labels on the colorbar
    ticks = np.linspace(0, 1, 5)  # You can adjust the number of ticks as needed
    tick_labels = [start_time_date + (end_time_date - start_time_date) * t for t in ticks]
    cbar.set_ticks(ticks)
    cbar.set_ticklabels([label.strftime("%Y-%m-%d") for label in tick_labels])    
    cbar.set_label("Time")

In [ ]:
def get_ticks(extent, step_size=0.2):
    # Calculate the start values for lat and long and generate sequences with step size of 0.2
    lat_start = np.ceil(extent[2] / step_size) * step_size
    lon_start = np.ceil(extent[0] / step_size) * step_size
    lats = np.arange(lat_start, extent[3], step_size)
    lons = np.arange(lon_start, extent[1], step_size)
    return lons, lats

def get_step_size(plot_extent):
    # calculate step size so that there are around 5 longitude labels
    
    step_size = (plot_extent[1]-plot_extent[0]) / 5
    if step_size <= 0.15:
        rounded_step_size = 0.1
    elif step_size <= 0.35:
        rounded_step_size = 0.2
    else:
        rounded_step_size = 0.5
    return rounded_step_size

def get_dem_extent(atr_dem):
    # get the extent which is required for plotting
    # [-156.0, -154.99, 18.99, 20.00]
    dem_extent = [float(atr_dem['X_FIRST']), float(atr_dem['X_FIRST']) + int(atr_dem['WIDTH'])*float(atr_dem['X_STEP']), 
        float(atr_dem['Y_FIRST']) + int(atr_dem['FILE_LENGTH'])*float(atr_dem['Y_STEP']), float(atr_dem['Y_FIRST'])] 
    return(dem_extent)     

def get_basemap(dem_file):
    dem, atr_dem = readfile.read(dem_file)
    dem_extent = get_dem_extent(atr_dem)
    ls = LightSource(azdeg=315, altdeg=45)
    dem_shade = ls.shade(dem, vert_exag=1.0, cmap=plt.cm.gray, vmin=-20000, vmax=np.nanmax(dem)+2500)
    return dem_shade,dem_extent

def plot_shaded_relief(ax, dem_file, plot_box = []):
    
    factory_default_figsize = plt.rcParamsDefault['figure.figsize']
    current_default_figsize = plt.rcParams['figure.figsize']
    increase_factor = 1.5
    if current_default_figsize == factory_default_figsize:
        new_default_figsize = [size * increase_factor for size in current_default_figsize]
        plt.rcParams['figure.figsize'] = new_default_figsize

    #plot DEM as background
    dem_shade, dem_extent = get_basemap(dem_file);

    ax.imshow(dem_shade, origin='upper', cmap=plt.cm.gray, extent=dem_extent)
    # ax.add_feature(cfeature.COASTLINE)
    
    if len(plot_box) == 0:
       plot_extent = dem_extent          # x_min, x_max, y_min, y_max
    else:
       plot_extent = [plot_box[2], plot_box[3], plot_box[0], plot_box[1]]   
     
    # Add latitude and longitude labels to the plot
    step_size = get_step_size(plot_extent)
    lons, lats = get_ticks(plot_extent, step_size = step_size)
    print(lons)
    print(lats)
    # ax.set_xticks(lons, crs=ccrs.PlateCarree())
    # ax.set_yticks(lats, crs=ccrs.PlateCarree())
    ax.set_xticks(lons)
    ax.set_yticks(lats)

    ax.xaxis.set_label_coords(0.5, -0.1)
    ax.yaxis.set_label_coords(-0.1, 0.5)

    # ax.set_extent(plot_extent)
    ax.set_xlim(plot_box[2], plot_box[3])
    ax.set_ylim(plot_box[0], plot_box[1])
    return ax

def get_GPS_old(GPS_list_file, plot_box):
    inf=open(GPS_list_file)
    next(inf)
    reader=csv.reader(inf, delimiter=' ')
    zipper=zip(*reader)
    gpslist = list(next(zipper))
    latlist = list(next(zipper))
    lonlist = list(next(zipper))
    new_gpslist = []
    new_latlist = []
    new_lonlist = []
    
    for i in range(0, len(gpslist)):
       gps = gpslist[i]
       lat = latlist[i]
       lon = lonlist[i]
       if float(lat) >= plot_box[0] and float(lat) <= plot_box[1] and float(lon) >= plot_box[2] and float(lon) <= plot_box[3]:
            new_gpslist.append(gps)
            new_latlist.append(lat)
            new_lonlist.append(lon)
   
    return new_gpslist, new_latlist, new_lonlist

def get_GPS(GPS_list_file, plot_box, start_time, end_time, unit, key_length):
    
    inf=open(GPS_list_file)
    next(inf)
    reader=csv.reader(inf, delimiter=' ')
    zipper=zip(*reader)
    gpslist = list(next(zipper))
    latlist = list(next(zipper))
    lonlist = list(next(zipper))
    new_gpslist = []
    new_latlist = []
    new_lonlist = []
    
    for i in range(0, len(gpslist)):
       gps = gpslist[i]
       lat = latlist[i]
       lon = lonlist[i]
       if float(lat) >= plot_box[0] and float(lat) <= plot_box[1] and float(lon) >= plot_box[2] and float(lon) <= plot_box[3]:
            new_gpslist.append(gps)
            new_latlist.append(lat)
            new_lonlist.append(lon)

    lat,lon,U,V,Z = get_quiver(new_gpslist, new_lonlist, new_latlist, start_time, end_time);
    duration_years, quiver_label = generate_quiver_label(unit, key_length, start_time, end_time)
    
    if unit == 'cm':
        U = [u * duration_years for u in U]
        V = [v * duration_years for v in V]
        Z = [z * duration_years for z in Z]

    return new_gpslist, lat, lon, U, V, Z, quiver_label

In [ ]:
def get_GPS_vel(sitename,time1,time2):
    filename='data/'+sitename+'.txt'
    dfin = read_csv(filename, header=0, delimiter=r"\s+")
    index = ['Time', 'East', 'North', 'Up']
    dataval=DataFrame(index=index);
    dataerr=DataFrame(index=index);
    dataval=concat([dfin['YYMMMDD'].rename('date'), (dfin['_e0(m)']+dfin['__east(m)']).rename('east'), (dfin['____n0(m)']+dfin['_north(m)']).rename('north'), 
                    (dfin['u0(m)']+dfin['____up(m)']).rename('up'),dfin['yyyy.yyyy'].rename('dateval')], axis=1)
    dataerr=concat([dfin['yyyy.yyyy'].rename('date'), dfin['sig_e(m)'], dfin['sig_n(m)'], dfin['sig_u(m)']], axis=1, 
                 ignore_index=False);
    dataval['date']=pd.to_datetime(dataval['date'], format='%y%b%d', errors='ignore')
    dataerr['date']=pd.to_datetime(dataval['date'], format='%y%b%d', errors='ignore')
    time1 = pd.to_datetime(time1)
    time2 = pd.to_datetime(time2)
    mask= (dataval['date'] > time1) & (dataval['date'] < time2)
    dataval=dataval[mask];dataerr=dataerr[mask];
    regr = linear_model.LinearRegression()
    regr.fit(dataval['dateval'].values.reshape(-1,1),dataval['east'].values.reshape(-1,1));east_vel=regr.coef_[0][0];
    regr.fit(dataval['dateval'].values.reshape(-1,1),dataval['north'].values.reshape(-1,1));north_vel=regr.coef_[0][0];
    regr.fit(dataval['dateval'].values.reshape(-1,1),dataval['up'].values.reshape(-1,1));up_vel=regr.coef_[0][0];
    return east_vel*1000, north_vel*1000,up_vel*1000;

def get_quiver(gpslist,lonlist,latlist,start_time,end_time):    
    #print time1,time2
    time1 = datetime.strptime(start_time, "%Y-%m-%d") 
    time2 = datetime.strptime(end_time, "%Y-%m-%d")
    u_ref, v_ref, z_ref = get_GPS_vel('MKEA', time1, time2)  #print u_ref,v_ref,z_ref
    X,Y,U,V,Z=[],[],[],[],[]      
    for i in range(len(gpslist)):
        try:
            u,v,z=get_GPS_vel(gpslist[i],time1,time2);u=u-u_ref;v=v-v_ref;
            U.append(float(u));V.append(float(v));Z.append(float(z));
            X.append(float(lonlist[i])),Y.append(float(latlist[i]));
        except:
            pass
    return X,Y,U,V,Z

def generate_quiver_label(unit, key_length, start_time, end_time):
    if unit == 'cm/yr':
        duration_years = 1
    if unit == 'cm':
        time1 = datetime.strptime(start_time, "%Y-%m-%d") 
        time2 = datetime.strptime(end_time, "%Y-%m-%d")
        duration = relativedelta(time2, time1)
        total_days = duration.days + (duration.months * 30) + (duration.years * 365)
        duration_years=total_days/365

    str_label = f"{key_length} {unit}"
    return duration_years, str_label

def generate_view_cmd(work_dir, date12, plot_box):
    ifgram_file = work_dir + '/geo_ifgramStack.h5'
    timeseries_file = work_dir + '/geo_timeseries_tropHgt_demErr.h5'
    geom_file = work_dir + '/geo_geometryRadar.h5'
    mask_file = work_dir + '/geo_maskTempCoh.h5'   # generated with generate_mask.py geo_geometryRadar.h5 height -m 3.5 -o waterMask.h5 option
    
    ## Configuration for InSAR background: check view.py -h for more plotting options.
    cmd = 'view.py {} unwrapPhase-{} -m {} -d {} '.format(ifgram_file, date12, mask_file, geom_file)
    cmd += f"--sub-lat {plot_box[0]} {plot_box[1]} --sub-lon {plot_box[2]} {plot_box[3]} "
    cmd += '--notitle -u cm -c jet_r --nocbar --noverbose '
    return cmd

In [ ]:
# Hard-wired and optional parameters
dem_file='data/demGeo.h5'
lines=sio.loadmat('./data/hawaii_lines_new.mat',squeeze_me=True);
GPS_list_file = 'data/GPS_BenBrooks_03-05full.txt'
work_dir = os.path.expanduser('~/offline/MaunaLoaSenDT87/mintpy/geo')
depth_range="0 10"
cmap_name = "plasma_r"; exclude_beginning = 0.2; exclude_end = 0.2
plot_box =[19.29, 19.6, -155.79, -155.41]

date12 = '20221216_20230202'
start_time, end_time, scale_fac, key_length, unit ="2022-09-01", "2022-11-26", 250, 4, "cm"

In [ ]:
# get dem, earthquake and GPS data, normalize event times for plotting)
dem_shade, dem_extent = get_basemap(dem_file)
events_df = get_earthquakes(start_time, end_time, plot_box)
norm_times = normalize_earthquake_times(events_df, start_time, end_time)
cmap = modify_colormap( cmap_name = cmap_name, exclude_beginning = exclude_beginning, exclude_end = exclude_end, show = False)

gps,lon,lat,U,V,Z,quiver_label = get_GPS(GPS_list_file, plot_box, start_time, end_time, unit, key_length)



In [ ]:
# Cell to plot GPS on shaded relief or interferogram
fig, ax = plt.subplots(figsize=[12, 5] )

plot_type = 'igram' 
plot_type = 'shaded_relief' 

if plot_type is 'igram':
    cmd = generate_view_cmd(work_dir, date12, plot_box)
    data, atr, inps = prep_slice(cmd)
    ax, inps, im, cbar = plot_slice(ax, data, atr, inps)
else:
    plot_shaded_relief(ax, dem_file, plot_box = plot_box)

# plot lines and events
ax.scatter(events_df["Longitude"],events_df["Latitude"],s=2*events_df["Magnitude"] ** 3,
         c=norm_times,cmap=cmap,alpha=0.8)
add_colorbar(cmap = cmap, start_time = start_time, end_time = end_time)
ax.set_title('Events:  ' + start_time + ' - ' + end_time);
quiv=ax.quiver(lon, lat, U, V, scale = scale_fac, color='blue')
ax.quiverkey(quiv, -155.50, 19.57, key_length*10 , quiver_label, labelpos='N',coordinates='data',
                  color='blue',fontproperties={'size': 20}) 

In [ ]:
# Cell to plot velocity from time series
dem_file='data/demGeo.h5'
lines=sio.loadmat('./data/hawaii_lines_new.mat',squeeze_me=True);
GPS_list_file = 'data/GPS_BenBrooks_03-05full.txt'
work_dir = os.path.expanduser('~/offline/MaunaLoaSenDT87/mintpy/geo')
depth_range="0 10"
cmap_name = "plasma_r"; exclude_beginning = 0.2; exclude_end = 0.2

plot_box =[19.29, 19.6, -155.79, -155.41]

date12 = '20221216_20230202'

start_time, end_time, scale_fac, key_length, unit ="2022-09-01", "2022-11-26", 250, 4, "cm"

# get shaded relief, earthquake and GPS data, normalize event times for plotting)
dem_shade, dem_extent = get_basemap(dem_file)
events_df = get_earthquakes(start_time, end_time, plot_box)

norm_times = normalize_earthquake_times(events_df, start_time, end_time)
cmap = modify_colormap( cmap_name = cmap_name, exclude_beginning = exclude_beginning, exclude_end = exclude_end, show = False)

gps,lon,lat,U,V,Z,quiver_label = get_GPS(GPS_list_file, plot_box, start_time, end_time, unit, key_length)

# # create figure
# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
plt.rcParams['font.size'] = 10

fig, ax = plt.subplots(figsize=[12, 5])

cmd = generate_view_cmd(work_dir, date12, plot_box)
data, atr, inps = prep_slice(cmd)

ax, inps, im, cbar = plot_slice(ax, data, atr, inps)
plt.rcParams['font.size'] = 12

# plot lines and events
ax.scatter(events_df["Longitude"],events_df["Latitude"],s=2*events_df["Magnitude"] ** 3,
         c=norm_times,cmap=cmap,alpha=0.8)
add_colorbar(cmap = cmap, start_time = start_time, end_time = end_time)
ax.set_title('Events:  ' + start_time + ' - ' + end_time);
quiv=ax.quiver(lon, lat, U, V, scale = scale_fac, color='blue')
ax.quiverkey(quiv, -155.50, 19.57, key_length*10 , quiver_label, labelpos='N',coordinates='data',
                  color='blue',fontproperties={'size': 20}) 

#set_printoptions(precision=2)
# max_index = U.index(max(U))
# print(U[max_index])
# print(V[max_index])